In [ ]:
import iris
import numpy as np
import holoviews as hv
import geoviews as gv
from cartopy import crs as ccrs
from cartopy import feature as cf

hv.notebook_extension()
%output widgets='live' size=200

<div class="alert alert-info" role="alert">
  <strong>Alert:</strong> The widgets in this notebook a set to render load dynamically and require a live notebook server. If you're viewing this notebook on a static website the widgets won't respond.
</div>

The main strength of [HoloViews](http://holoviews.org) and its extensions (like GeoViews) is the ability to quickly explore complex datasets by declaring lower dimensional views into a higher-dimensional space. In HoloViews we refer to the interface that allows you to do this as the conversion API. To begin with we will load a multi-dimensional dataset of surface temperatures using [Iris](http://scitools.org.uk/iris/):

In [ ]:
import numpy as np
import iris
import iris.coords

def realization_metadata(cube, field, fname):
    if not cube.coords('realization'):
        realization_number = fname[-6:-3]
        realization_coord = iris.coords.AuxCoord(np.int32(realization_number), 'realization')
        cube.add_aux_coord(realization_coord)

surface_temp = iris.load_cube(iris.sample_data_path('GloSea4', 'ensemble_???.pp'),
              iris.Constraint('surface_temperature', realization=lambda value: True),
              callback=realization_metadata)

As we saw in the [Introductory Tutorial](Introductory_Tutorial.ipynb) we can simple wrap this iris Cube datastructure in a HoloViews Dataset.

In [ ]:
cube = hv.Dataset(surface_temp)
cube

From the repr we can immediately see the list of key dimensions (time, realization, longitude and latitude) and the value dimension of the cube (surface_temperature), however unlike most other HoloViews Elements the ``Dataset`` Element does not display itself visually. This is because it can be n-dimensional and therefore does not have a straightforward visual representation. To view the cube we first have to transform it into individually visualizable chunks.

# Conversions

A HoloViews Dataset is a wrapper around a complex multi-dimensional datastructure which allows the user to convert their data into individually visualizable views, which are usually of lower dimensionality. This is done by grouping the data by some dimension and then casting it to a specific Element type, which visualizes itself.

The ``dataset.to`` interface makes this especially easy, you supply the Element type that you want to view the data as and the key dimensions of that view and it will figure out the rest. Depending on the type of Element you can specify one or more dimensions to be displayed. This means that only certain Elements allow you to display the data on a cartographic projection. Recall that the cube we are working with has 4 coordinate dimensions (or key dimensions as they are known in HoloViews) - time, realization, longitude and latitude. A geographic plot is defined as a plot that has longitude along the x-axis and latitude along the y-axis.

To declare a two dimensional plot type we therefore simply request an Image plot of ``longitude`` and ``latitude``. All the remaining dimensions are automatically inferred, i.e. the value dimension (vdim) is ``surface_temperature`` and any remaining dimensions are assigned to the ``HoloMap`` datastructure, which allows you to explore the data using widgets:

In [ ]:
geo_dims = ['longitude', 'latitude']
coastline = gv.Feature(cf.COASTLINE)
cube.to(gv.Image, geo_dims) * coastline

In this way we can visualize the 2D data in a number of ways, currently either as an ``Image`` (as above) or as ``LineContours``, ``FilledContours`` or ``Points``:

In [ ]:
%%opts Layout [fig_inches=(4, 8)]
%%opts Points [color_index=2 size_index=None colorbar=True]
hv.Layout([cube.to(el, geo_dims) * coastline
           for el in [gv.FilledContours, gv.LineContours, gv.Points]]).cols(1)

Note that by default the conversion interface will automatically expand all the individual Elements, which can take some time if the data is very large. Instead we can also request the objects to be expanded dynamically using the ``dynamic`` keyword:

In [ ]:
cube.to(gv.Image, geo_dims, dynamic=True) * coastline

This means that the data for each frame is only extracted when you're actually viewing that part of the data, which can have huge benefits in terms of speed and memory consumption.

## Non-geographic views

So far we have focused entirely on geographic views of the data, plotting the data on a projection. However the conversion interface is completely general and we can slice and dice the data in any way we like. The simplest example of such a view is simply a view showing the temperature over time for each realization, longitude and latitude coordinate:

In [ ]:
%%opts Curve [xrotation=25 aspect=2]
cube.to(hv.Curve, 'time', dynamic=True)

Another possible view is to the data as a ``HeatMap`` over time and realization at each longitude and latitude:

In [ ]:
%%opts HeatMap [show_values=False colorbar=True]
cube.to(hv.HeatMap, ['realization', 'time'], dynamic=True)

## Lower dimensional views

So far all the conversions have been seen have encapsulated all the available coordinate dimensions. However often times we want to see the spread of values along one or more dimensions. A simple example of this is a box plot where we might want to see the spread of surface_temperature on each day but don't want to ignore the latitude and longitude coordinates. To ignore particular dimensions we can explicitly declare the map dimensions (i.e. the key dimensions of the HoloMap container). By declaring an empty list of ``mdims`` for instance we can tell the conversion interface to simply ignore all dimensions except the particular key dimension that was supplied in this case the 'time' and 'realization':

In [ ]:
%%opts BoxWhisker [xrotation=25 bgcolor='w']
hv.Layout([cube.to.box(d, mdims=[]) for d in ['time', 'realization']])

This also gives us access to other statistical plot types, e.g. with the ``seaborn`` library installed we also have access to the ``Distribution`` Element which visualizes the data as a kernel density estimate. In this way we can visualize how the distribution of surface temperature values varies over time and the model realizations. We do this by ignoring the 'latitude' and 'longitude' in the list of mdims, generating a lower dimensional view into the data and laying out 'time' and 'realization' in a GridSpace.

In [ ]:
%opts GridSpace [shared_xaxis=True] 
%opts Distribution [bgcolor='w' show_grid=False xticks=[220, 300]]
try:
    import seaborn
    grid = cube.to.distribution(mdims=['realization', 'time']).grid()
except:
    grid = None
grid

## Reducing the data

So far all the examples we have seen have displayed all the data in some way or another. Another way to explore a dataset is to explicitly reduce the dimensionality or select subregions of a dataset. There are two main ways to do this, either we explicitly select a subset of the data or we collapse a dimension using an aggregation function, e.g. by computing a mean along a particular dimension.

### Selecting slices

Using the ``select`` method we can easily select ranges of coordinates in the Cube. The select method does not however know that latitude and longitude are cyclic, so instead we can select regions at both ends of the prime meridian ($0^\circ$ longitude) and overlay them. In this way we can stitch together multiple cubes or simply view a subregion specific subregions:

In [ ]:
northern = cube.select(latitude=(25, 75))
(northern.select(longitude=(260, 305)).to(gv.Image, geo_dims) *
 northern.select(longitude=(350, 360)).to(gv.Image, geo_dims) *
 northern.select(longitude=(0, 50)).to(gv.Image, geo_dims) *
 coastline)

### Selecting a particular coordinate

We can select a particular coordinate using the select method, cast the data to Curves, reindex the data to drop the now constant latitude and longitude dimensions and overlay the remaining 'realization' dimension.

In [ ]:
%opts NdOverlay [xrotation=25 aspect=2  legend_position='right' legend_cols=2] Curve (color=Palette('Set1'))
cube.select(latitude=0, longitude=0).to(hv.Curve, ['time']).reindex().overlay()

### Collapsing coordinates

Alternatively we can also collapse specific coordinates on the iris Cube first, then wrap it in a Dataset and convert it to curves indexed over time, which we can again overlay:

In [ ]:
for dim in ['longitude', 'latitude']:
    if cube.data.coord(dim).bounds is None:
        cube.data.coord(dim).guess_bounds()

grid_weights = iris.analysis.cartography.area_weights(cube.data)
collapsed_cube = cube.data.collapsed(['longitude', 'latitude'], iris.analysis.MEAN, weights=grid_weights)
hv.Dataset(collapsed_cube).to(hv.Curve, 'time').overlay()